In [1]:
# Importing thr SQL and required pakages
from tabulate import tabulate
import mysql.connector


# building the connection between MYSQL and PYTHON JUPYTER notebook
connection = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'root',
    database = 'hospital_database'
)

# Check the connection is connected before performing the 
if connection.is_connected():
    cursor = connection.cursor()
    print("The connection is established")

# function declaration for the details of adding a new doctor into the hospital management.

def rec_addopd():
    print("Adding the new OPD: ")
    
    try:
        # take input from the user to get details
        
        opdno = int(input('Enter the OPD number: '))

        # check if opdno already exists in the "opd" table
        
        query = "SELECT * FROM opd WHERE opdno = %s"
        cursor.execute(query, (opdno,))
        result = cursor.fetchone()
        
        # if the given opdno is already there in the data base fetch that details
        
        if result:
            print("Duplicates are not allowed. OPD number already exists.")
        
        # if the given opdno is not there in the database then continue the process
        
        else:
            cursor.execute("SELECT doa FROM appoint WHERE opdno = %s", (opdno,))
            result = cursor.fetchone()
            
            # fetching the zeroth index for the result
            doa = result[0] if result else input("Enter Appointment Date in the format(YYYY-MM-DD): ")

            # fetching the patient details
            cursor.execute("SELECT pname FROM opd WHERE opdno=%s", (opdno,))
            result = cursor.fetchone()
            
            # fetching the zeroith index for the result (Patient_Name)
            pname = result[0] if result else input("Enter patient name: ")
            cursor.execute("SELECT padd, pmob FROM patient WHERE pname = %s", (pname,))
            result = cursor.fetchone()
            
            padd, pmob = result if result else (input("Enter patient address:"), input("Enter patient mobile number: "))

            print("The Doctor Details: ")

            try:
                query = "SELECT did, dname, department FROM doctor"
                cursor.execute(query)
                results = cursor.fetchall()

                if not results:
                    print("No doctors available.")
                    return

                headers = ["did", "dname", "department"]
                table = tabulate(results, headers=headers, tablefmt="pretty")
                print(table)

                # ask for the user input(doctor ID [did])
                
                did = int(input("Enter the doctor ID: "))
                dname = None 

                query = "SELECT * FROM doctor WHERE did = %s"
                cursor.execute(query, (did,))
                res = cursor.fetchall()

                if res:
                    headers = ["did","dname","department"]
                    print(tabulate(res, headers=headers, tablefmt="pretty"))
                    dname = res[0][1]
                else:
                    print("No records for the selected doctor ID.")
                    return
                
                symptom = input("enter the symptom: ").lower()
                fee = input("Enter the fee: ")

                # performing the SQL query into opd table
                query = "INSERT INTO opd(opdno, pname, dname, fee) VALUES (%s, %s, %s, %s);"
                values = (opdno, pname, dname, fee)
                cursor.execute(query, values)
                connection.commit()
                print("OPD record inserted successfully")

                # insert the data into appoint table
                query = "INSERT INTO appoint(opdno, pname, dname,doa, symptom) VALUES (%s, %s, %s, %s, %s);"
                values = (opdno, pname, dname, doa, symptom)
                cursor.execute(query, values)
                connection.commit()
                print("Appointment record inserted successfully")

                # performing the query to insert data into the patient table
                query = "INSERT INTO patient (pname, padd, pmob) VALUES (%s, %s, %s);"
                values = (pname, padd, pmob)
                cursor.execute(query, values)
                connection.commit()
                print("Patient record inserted successfully")

                # OPD details
                query = "SELECT * FROM opd"
                cursor.execute(query)
                results = cursor.fetchall()

                print("The OPD records are: ")
                
                if results:
                    headers = ["opdno", "pname", "dname", "fee"]
                    table = tabulate(results, headers=headers, tablefmt="pretty")
                    print(table)
                else:
                    print("Invalid Doctor id. Please enter a valid Doctor ID: ")

            except ValueError as val_err:
                print(f"Invalid input: {val_err}")

            except mysql.connector.Error as err:
                print(f"MySql Error is: {err}")

            except Exception as e:
                print(f"Unknown error is: {e}")
                    
    except ValueError as val_err:
        print(f"Invalid input: {val_err}")

    except mysql.connector.Error as err:
        print(f"MySql Error is: {err}")

    except Exception as e:
        print(f"Unknown error is: {e}")
        
def rec_editopd():
    print("You are modifying or Updating the existing data: ")
    
    try:      
        # take the inputs from the user to modify the existing data
        
        opdno = int(input("enter the opdno: "))
        
        # check if opdno already exists in the "opd" table
        
        query = "select * from opd where opdno = %s"
        cursor.execute(query, (opdno,))
        re = cursor.fetchone()
        
        
        if not re:
            print("duplicates not allowed. opdno already exists.")
            return
        # if that opdno exists, ask for the new fee
        
        fee = int(input("Enter the Updated fee: "))
        
        query = "UPDATE opd SET fee = %s WHERE opdno = %s"
        
        values = (fee, opdno)
        
        cursor.execute(query,values)
        
        connection.commit()
        
        print("Updated Successfully.")
               
    except ValueError as val_err:
        print(f"Invalid input:{val_err}")                    
                    
    except mysql.connector.Error as err:
        print(f"Mysql Error is: {err}")
        
    except Exception as e:
        print(f"unknown error is: {e}")      
        
def rec_deleteopd():
    print("Deleting the details of opd: ")
    
    try:
        show = "select * from opd"
        cursor.execute(show)
        r = cursor.fetchall()
        print(tabulate(r,headers=["opdno","pname","dname","fee"],tablefmt = "pretty"))
        
        opdno = int(input("enter the opdno to delete: "))
        
        # please check if the opdno exist befor deleting
        
        check = "select opdno from opd where opdno = %s"
        
        cursor.execute(check,(opdno,))
        
        res = cursor.fetchone()
        
        if res:
            # ask for the confirmation for the deletion of opd data
            
            c = input("Are you sure want to delete. (y/n): ").lower()
            if c=='y':
                query = "delete from opd where opdno = %s"
                cursor.execute(query,(opdno,))
                connection.commit()
                print("deleted successfully")
        

                show = "select * from opd"
                cursor.execute(show)
                r = cursor.fetchall()
                print(tabulate(r,headers=["opdno","pname","dname","fee"],tablefmt = "pretty"))
            else:
                print("cancel")
        else:
            print("No record")
            
    except ValueError as val_err:
        print(f"Invalid input:{val_err}")
            
    except mysql.connector.Error as err:
        print(f"Mysql Error is: {err}")  
        
    except Exception as e:
        print(f"Unknown error is: {e}")
        
def rec_selectopd():
    print("The OPD details: ")
    
    try:
        query = "SELECT opdno, pname, dname, fee FROM opd"
        cursor.execute(query)
        results = cursor.fetchall()
        
        if not results:
            print("No opd records found.")
            return
        
        headers = ["opdno", "pname", "dname", "fee"]
        table = tabulate(results, headers=headers, tablefmt="pretty")
        print(table)
        
        # ask for the user input
        opdno = int(input("enter the opdno to select: "))
        
        # sql query
        query = "SELECT * FROM opd WHERE opdno = %s"
        cursor.execute(query, (opdno,))
        resu = cursor.fetchall()
        
        if resu:
            headers = ["opdno", "pname", "dname", "fee"]
            values = resu
            print(tabulate(resu, headers=headers, tablefmt="pretty"))
        else:
            print("No records found for the opdno.")
            
    except ValueError as val_err:
        print(f"Invalid input: {val_err}")
        
    except mysql.connector.Error as err:
        print(f"Mysql Error is: {err}")
        
    except Exception as e: 
        print(f"Unknown error is: {e}")


The connection is established
